In [2]:
import pandas as pd
from openai import OpenAI
import openai
from openai.types.chat.completion_create_params import ResponseFormat
import json
import csv
import re
import os
import numpy as np
from sentence_transformers import SentenceTransformer
import openai 
from openai import AzureOpenAI

c:\Users\elena\articles_gpt\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
api_key = #azure openai api key
azure_endpoint = # azure endpoint

In [4]:
client = AzureOpenAI(
    api_key=api_key,
    api_version="2024-03-01-preview",
    azure_endpoint=azure_endpoint
)
deployment_gpt_name = # deployment name

In [6]:
with open('abstract_policies.json', 'r', encoding='utf=8') as file:
    abstracts_json = json.load(file)

In [7]:
with open('search_data_simple_GDPR.txt', 'r') as file:
    codes_questions = file.readlines()

codes_questions
questions = [item.strip('\n\t') for item in codes_questions]
questions
list_questions = {}
 
for quest in range(0, len(questions), 2):
    code = questions[quest].strip(':')
    question = questions[quest+1]
    list_questions[code] = question

In [8]:
def save_response_trial(policy_id, response, file_path):
    response_content = response.choices[0].message.content if response.choices else "No response from the API"
    with open(file_path, "r") as file:
        existing_data = json.load(file)
    
    existing_data[policy_id] = response_content

    with open(file_path, "w") as file:
        json.dump(existing_data, file, indent=4)

In [9]:
def prompt_format(questions_dict):
    messages = [
    {"role": "system", "content": """ You are an auditor that needs to review multiple privacy policies, each one identified with a `PolicyID`.
    For each policy, I am going to provide you the `PolicyID` and `PolicyText`. For each policy, answer each question that I give you with a "Yes"/"No" answer and if the answer is "Yes" then provide an extract from the policy that is LONGER than 200 characters and best fits the answer, otherwise return an empty string, nothing else that differs from this.
    Make sure that your response is only in a JSON format like this and DO NOT PROVIDE ANY ADDITIONAL TEXT: `{'QuestionID': {"Answer": "Your answer", "Extract": "Extract from the policy"}}`, where " Your Answer" represents your answer to the question, "Extract from the policy" is the best fit extract and 'QuestionID' is the id of the question that I provide you with.
    Below is the list of questions, in the following format `{'QuestionID':'QuestionText'}`:"""+ f"{questions_dict}" }
]
    return messages

In [10]:
def get_response_by_policy_id(data, policy_id):
    if len(data) > 0:
        for response in data["responses"]:
            if response["PolicyID"] == policy_id:
                return response["Response"]
    return None

In [11]:
def get_non_cached_questions(cached_questions, original_questions):
    missing = {}
    for key, value in original_questions.items():
        if cached_questions:
            if key not in cached_questions.keys():
                missing[key] = value 
        else:
            return original_questions

    return missing

In [12]:
# change number of questions and policies as needed
questions_for_answering = dict(list(list_questions.items()))
test_policies = {}

for key, value in list(abstracts_json.items()):
    test_policies[key] = value

In [15]:
responses = {}
file_path = '' # save responses to a file
answered_policies_ids = []
with open(file_path, "r") as file:
    response_data = json.load(file)
if len(response_data) > 0:
    for policy in response_data['responses']:
        answered_policies_ids.append(policy['PolicyID'])

def add_message(role, content, message_format):
    message_format.append( {"role": role, "content": content})



def handle_conversation():
    for policy_id, policy in test_policies.items():
        prompt = "{'PolicyID':'"+f"{policy_id}"+"', 'PolicyText':'"+ policy +"'}."
        cached_answers = get_response_by_policy_id(response_data, policy_id)
        missing_questions = get_non_cached_questions(cached_answers, questions_for_answering)

        if len(missing_questions) > 0:
    
            message_format = prompt_format(missing_questions)
            add_message("user", prompt, message_format)

            messages_for_api = [{"role": m["role"], "content": m["content"]} for m in message_format]

            message_format.remove({"role": "user", "content": prompt})

            response = client.chat.completions.create(
                model=deployment_gpt_name,
                messages=messages_for_api
            )
            
            responses[policy_id] = response

            if response.choices and len(response.choices) > 0:
                full_response = response.choices[0].message.content
                save_response_trial(policy_id, response, file_path)
                response = None
            else:
                print("No response from the API")
handle_conversation()